In [37]:
from numpy import dot
from numpy.linalg import norm
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [38]:
merged = pd.read_pickle("../Data/merged.pkl")

In [40]:
merged

,Автотранспорт,Алкогольный магазин,"Бар ""Иной""","Бар ""Коктейль""",Барбершоп,Не определено,Перевод денег на депозит,Покупка одежды,Покупка техники,Покупки Ozon,"Ресторан ""Буйвол""",Салон красоты,Такси,Турагенство,Сумма расходов
ID Клиента,,,,,,,,,,,,,,,
10157447,2,16,0,11,26,17,0,2,0,0,11,0,9,13,-0.259226
10157454,0,0,0,0,0,0,0,0,0,0,0,0,1,0,-0.269286
10158439,0,0,0,9,11,0,0,0,5,0,2,0,0,0,-0.228334
10158854,6,44,0,61,56,44,0,2,53,7,95,5,89,66,-0.218408
10159585,6,4,0,2,0,11,0,3,3,8,0,3,1,26,-0.263764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11328050,0,9,0,37,15,27,0,4,29,2,15,0,19,29,-0.193631
11328121,0,2,0,0,0,2,0,4,0,0,0,0,5,0,-0.046777
11335636,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.279330


##  Select the Target Group

#### Which sector would you like to improve?
By choosing, for example, the Gastronomic sector, you select a group of customers who show high attendance and spendings, having a discount above 15%.
Further, the recommender system looks for similar customers who can be offered discounts for the selected sector and the indicators will increase.

In [67]:
# # Attendance
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
          [11018820, 11127430, 11259771, 11260095]]

In [74]:
# # Spendings
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#           [10574034, 11018820, 11127430, 11259771, 11260095]]

In [148]:
# Sector: Gastronomic
ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
          [11127430, 10355641, 11259771, 11018820, 11260095, 10356174]]

In [100]:
# # Sector: Tourism
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#           [11127430, 10355641, 10562323, 11259771, 11018820, 11260095]]

In [107]:
# # Sector: Shopping
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#     [11127430, 10355641, 11259771, 11018820, 10359664, 10409000, 10830303, 10216655, 10201728]]

In [116]:
# # Sector: Auto
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#           [11127430, 10355641, 11128132, 11259771, 11018820, 10830303]]

In [133]:
# # Sector: Beauty
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#           [11127430, 10355641, 10574034, 11259771, 11018820, 10358256]]

In [147]:
# # Sector: Other
# ideals = [list(map(float,merged.loc[i].to_numpy(dtype=float))) for i in 
#           [11127430, 10355641, 11259771, 11018820, 10356174]]

In [149]:
ideals

[[31.0,
  40.0,
  4.0,
  180.0,
  29.0,
  96.0,
  0.0,
  12.0,
  8.0,
  16.0,
  45.0,
  5.0,
  24.0,
  58.0,
  0.14081983824634633],
 [41.0,
  413.0,
  42.0,
  138.0,
  380.0,
  442.0,
  17.0,
  210.0,
  159.0,
  774.0,
  520.0,
  255.0,
  228.0,
  327.0,
  -0.21406400093632258],
 [76.0,
  227.0,
  0.0,
  595.0,
  682.0,
  582.0,
  2.0,
  94.0,
  85.0,
  34.0,
  309.0,
  192.0,
  518.0,
  317.0,
  -0.023509211676995205],
 [0.0,
  65.0,
  0.0,
  184.0,
  167.0,
  164.0,
  0.0,
  60.0,
  2.0,
  26.0,
  87.0,
  0.0,
  69.0,
  232.0,
  0.5991405996547979],
 [0.0,
  2.0,
  0.0,
  145.0,
  47.0,
  0.0,
  0.0,
  0.0,
  11.0,
  8.0,
  25.0,
  0.0,
  0.0,
  307.0,
  0.16314841134827296],
 [4.0,
  11.0,
  2.0,
  81.0,
  113.0,
  126.0,
  0.0,
  13.0,
  12.0,
  0.0,
  81.0,
  0.0,
  8.0,
  28.0,
  -0.20801244617736772]]

In [150]:
merge_vec = list(map(list,merged.to_numpy(dtype=float)))

In [151]:
clients = {key:val for val,key in zip(merge_vec,merged.index.values.tolist())}

In [152]:
class Recommender:
    def __init__(self, ideal):
        self.clients_vector = None
        self.ideal_clients = ideal
          
    def cos_sim(self,a,b):
        similarity = dot(a, b)/(norm(a)*norm(b))
        return similarity
    
    def collaborative_filtering(self,user):
        count = 0
        for i in self.ideal_clients:
            similarity = self.cos_sim(i,user)
            print(similarity)
            if 0.9 <= similarity < 1:
                count += 1
        return count >= 2

    def recommend_clients(self, clients):
        valid_clients = []
        for client_id, client in clients.items():
            if self.collaborative_filtering(client):
                valid_clients.append(client_id)
        return valid_clients



In [153]:
recommender = Recommender(ideals)
Clients = recommender.recommend_clients(clients)

0.6965475312676986
0.7042038806001961
0.9213313718615371
0.889246933330333
0.49863928921204587
0.8720589769756846
0.1019693571499028
0.16909938122176355
0.37989561662603627
0.1644890959455402
-0.0001233413357115693
0.03748313680899251
0.5998541286018593
0.36660987571664116
0.6945298417402799
0.5985892423999052
0.3685796983837788
0.6955180711274751
0.6976861189676429
0.7162129630378895
0.8645143392251837
0.8044576070071267
0.541819355925424
0.7277853062529829
0.5013096151168398
0.5952402453074757
0.4719057752753682
0.7185754684566555
0.7922595444708918
0.3804215333498408
0.7615271654468608
0.22380467235851903
0.6316372681463107
0.5879819131805608
0.435229338536377
0.5887956394368543
0.5312589515039682
0.5283704085160746
0.6377822703638452
0.6560593668304184
0.3416890091703774
0.6695381361756344
0.42433953746916686
0.3426146943242251
0.44620428399254003
0.409811531858165
0.000972109421825656
0.6114296903919563
0.4232508503864337
0.4900278583187554
0.6217338630188004
0.5323672668428403
0.

0.3903040147174788
0.8459777395248301
0.27012952093326337
0.6812956451712922
0.6769443255572224
0.5326402454134079
0.6413616573459165
0.9457715998510609
0.6153784060537195
0.8752454004768082
0.9193054663235353
0.6790779625521781
0.8127574773381981
0.8379875298408238
0.17983474545219072
0.5079730689806605
0.5046101136652464
0.42345462441608145
0.44020745046821946
0.7964384300423626
0.11296946455714854
0.4650398151657517
0.4644571789055886
0.4240182545218323
0.40065906898163717
0.7190008164420656
0.6727343699233344
0.7903715601847611
0.893598940135843
0.7707954107921715
0.6895750581931044
0.3806913951194013
0.30511531939877873
0.39745722240207726
0.36556678261347636
0.00020756743879085044
0.5454863470999364
0.8096241311633611
0.14611392550627045
0.49445740557849044
0.4905800687015506
0.43238435763356825
0.44375312112213583
0.2811670650432783
0.3759584851866061
0.503399095781359
0.7072380592617502
0.7960583901251073
0.4346353814042976
0.34994169735832187
0.19893558401422404
0.514375756494

0.8348217448024253
0.7871080951953313
0.579536410379583
0.5808336211666071
0.653786542339331
0.6612004599574232
0.5988732802019858
0.34328437943666623
0.554335091753665
0.5777974144604333
0.40033369966864746
0.47853989488588405
0.7268147945392311
0.6955109931503033
0.43900411602880285
0.07486886367121852
0.2970830808581832
0.3519881995439253
0.18001439427978566
0.05952070511292546
0.23648834047006817
0.4792983563943404
0.3668053647042903
0.5199736884279372
0.45206875213423414
0.024489109300691016
0.6279277196245636
0.7816308703572848
0.6530094720911487
0.9318853545849407
0.9411165801082317
0.679239500309423
0.8682831058884553
0.49027599509576847
0.6809025630293137
0.7539067391660152
0.7010767183594525
0.4339160431781585
0.5392772294641413
0.17572867018222485
0.3162409264750845
0.1719143124910822
0.1602756580277238
0.005762311480352519
0.05292794028583713
0.4229060192044942
0.3392890960542961
0.44187279316860434
0.4056490443631574
-1.2757680488062787e-05
0.6069460525358086
0.20354357019

0.12779228512790475
0.29174067152415345
0.5178858245917808
0.4132111168597885
0.13663408808299216
0.5443765269854396
0.6960261504824298
0.5854679147256133
0.6843149881944802
0.8472619084472464
0.8139857692349566
0.5413753353669585
0.6303283558929592
0.62656038968025
0.713345784785058
0.5847374825993827
0.1730862259210698
0.7565815665695794
0.9776259847113515
0.5766925750637045
0.8286304456816799
0.8172697292173582
0.5131572476619125
0.8298863764153189
0.28728672228625585
0.4479651169350975
0.6871808836750589
0.5467279026759926
0.17375471579141857
0.656340472258856
0.5381571055080809
0.4153941508967253
0.708866517315064
0.5101522797176826
0.24404628085141922
0.4860120594455248
0.4113628253935948
0.32985519863373947
0.42963783098616587
0.39480870766234955
0.00011152135442535983
0.5898849421092612
0.408349976691187
0.3277661403982608
0.4268199396738196
0.3914791989424772
-0.00012338920658172837
0.5864987543010647
0.7067014516544179
0.5507121431836937
0.7547970341412742
0.7140550642336609


0.2579991805841225
0.6193173740548564
0.4218684376927023
0.33848793301387337
0.44082013507302414
0.4046119067696593
-3.512271531903241e-05
0.6055463110681679
0.1019693571499028
0.16909938122176355
0.37989561662603627
0.1644890959455402
-0.0001233413357115693
0.03748313680899251
0.4229855704539629
0.3393478401712852
0.44195080312625823
0.405732153734764
-9.122666006285029e-06
0.607045707484324
0.4257494704261789
0.3450533502320323
0.4450339931762333
0.40791696141076145
0.0004899591134756643
0.6084762380122059
0.10196435342439852
0.1690913033857616
0.3798774043219758
0.1644809355546791
-0.0001234228633664045
0.03748152430117657
0.10196435342439852
0.1690913033857616
0.3798774043219758
0.1644809355546791
-0.0001234228633664045
0.03748152430117657
0.4083409407251962
0.3277589436126116
0.4268105513659213
0.3914704612307176
-0.0001234266028857365
0.586485936388369
0.7681989706014769
0.10268160259171333
0.4377174376708539
0.4406589677627347
0.4082227156959624
0.3782946173399778
0.423076194426

### The recommender system brought out the customers most susceptible to take advantage of the proposed large discount, thereby increasing the attendance and revenue indicators for the selected sector.

In [154]:
print(len(Clients))
Clients

21


[10216816,
 10217384,
 10218209,
 10218864,
 10219380,
 10354725,
 10355392,
 10361741,
 10402701,
 10404735,
 10574677,
 10583558,
 10837991,
 11008961,
 11011968,
 11012574,
 11018939,
 11135326,
 11257217,
 11259088,
 11259804]

In [156]:
# Check the client attendance
merged.loc[10216816]

Автотранспорт                 0.000000
Алкогольный магазин          85.000000
Бар "Иной"                    0.000000
Бар "Коктейль"              183.000000
Барбершоп                   295.000000
Не определено               125.000000
Перевод денег на депозит      0.000000
Покупка одежды               24.000000
Покупка техники              40.000000
Покупки Ozon                 26.000000
Ресторан "Буйвол"           169.000000
Салон красоты                 0.000000
Такси                        76.000000
Турагенство                  83.000000
Сумма расходов               -0.261918
Name: 10216816, dtype: float64